# Movies Clustering

- Ricardo Méndez 21289
- Sara Echeverría 21371
- Sebastián Juárez 21471

Enlace al repositorio: https://github.com/bl33h/moviesClustering

In [9]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score
plt.style.use('ggplot')

In [10]:
df = pd.read_csv('data/movies.csv', encoding='latin1')
df.head()

,id,budget,genres,homePage,productionCompany,productionCompanyCountry,productionCountry,revenue,runtime,video,...,popularity,releaseDate,voteAvg,voteCount,genresAmount,productionCoAmount,productionCountriesAmount,actorsAmount,castWomenAmount,castMenAmount
0,5,4000000,Crime|Comedy,https://www.miramax.com/movie/four-rooms/,Miramax|A Band Apart,US|US,United States of America,4257354.0,98,False,...,20.880,1995-12-09,5.7,2077,2,2,1,25,15,9
1,6,21000000,Action|Thriller|Crime,NaN,Universal Pictures|Largo Entertainment|JVC,US|US|JP,Japan|United States of America,12136938.0,110,False,...,9.596,1993-10-15,6.5,223,3,3,2,15,3,9
2,11,11000000,Adventure|Action|Science Fiction,http://www.starwars.com/films/star-wars-episod...,Lucasfilm|20th Century Fox,US|US,United States of America,775398007.0,121,NaN,...,100.003,1977-05-25,8.2,16598,3,2,1,105,5,62
3,12,94000000,Animation|Family,http://movies.disney.com/finding-nemo,Pixar,US,United States of America,940335536.0,100,NaN,...,134.435,2003-05-30,7.8,15928,2,1,1,24,5,18
4,13,55000000,Comedy|Drama|Romance,NaN,Paramount|The Steve Tisch Company,US|,United States of America,677387716.0,142,False,...,58.751,1994-07-06,8.5,22045,3,2,1,76,18,48


### 1. Haga el preprocesamiento del dataset, explique qué variables no aportan información a la generación de grupos y por qué. Describa con qué variables calculará los grupos.

In [12]:
print(df.columns)

Index(['id', 'budget', 'genres', 'homePage', 'productionCompany',
       'productionCompanyCountry', 'productionCountry', 'revenue', 'runtime',
       'video', 'director', 'actors', 'actorsPopularity', 'actorsCharacter',
       'originalTitle', 'title', 'originalLanguage', 'popularity',
       'releaseDate', 'voteAvg', 'voteCount', 'genresAmount',
       'productionCoAmount', 'productionCountriesAmount', 'actorsAmount',
       'castWomenAmount', 'castMenAmount'],
      dtype='object')


In [13]:
features = df[['budget', 'genres', 'homePage', 'productionCompany','productionCompanyCountry', 'productionCountry', 'revenue', 'runtime','video', 'director', 'actors', 'actorsPopularity', 'actorsCharacter','originalTitle', 'title', 'originalLanguage', 'popularity','releaseDate', 'voteAvg', 'voteCount', 'genresAmount','productionCoAmount', 'productionCountriesAmount', 'actorsAmount','castWomenAmount', 'castMenAmount']].copy()

In [14]:
df.describe()

,id,budget,revenue,runtime,popularity,voteAvg,voteCount,genresAmount,productionCoAmount,productionCountriesAmount,actorsAmount,castWomenAmount,castMenAmount
count,10000.000000,1.000000e+04,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,9838.000000,9478.000000
mean,249876.829300,1.855163e+07,5.673793e+07,100.268100,51.393907,6.483490,1342.381800,2.596500,3.171400,1.751000,2147.666600,7.148201,14.119434
std,257380.109004,3.662669e+07,1.495854e+08,27.777829,216.729552,0.984274,2564.196637,1.154565,2.539738,3.012093,37200.075802,6.281767,13.131693
min,5.000000,0.000000e+00,0.000000e+00,0.000000,4.258000,1.300000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,12286.500000,0.000000e+00,0.000000e+00,90.000000,14.577750,5.900000,120.000000,2.000000,2.000000,1.000000,13.000000,3.000000,6.000000
50%,152558.000000,5.000000e+05,1.631245e+05,100.000000,21.905500,6.500000,415.000000,3.000000,3.000000,1.000000,21.000000,6.000000,11.000000
75%,452021.750000,2.000000e+07,4.479661e+07,113.000000,40.654000,7.200000,1316.000000,3.000000,4.000000,2.000000,36.000000,9.000000,19.000000
max,922260.000000,3.800000e+08,2.847246e+09,750.000000,11474.647000,10.000000,30788.000000,16.000000,89.000000,155.000000,919590.000000,106.000000,626.000000


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         10000 non-null  int64  
 1   budget                     10000 non-null  int64  
 2   genres                     9947 non-null   object 
 3   homePage                   4193 non-null   object 
 4   productionCompany          9543 non-null   object 
 5   productionCompanyCountry   8720 non-null   object 
 6   productionCountry          9767 non-null   object 
 7   revenue                    10000 non-null  float64
 8   runtime                    10000 non-null  int64  
 9   video                      9514 non-null   object 
 10  director                   9926 non-null   object 
 11  actors                     9920 non-null   object 
 12  actorsPopularity           9913 non-null   object 
 13  actorsCharacter            9953 non-null   obje

A criterio propio y considerando tanto el tipo de dato de cada una de las variables como su papel en el dataset, es posible clasificarlas de la siguiente manera:

- Variables no útiles para clustering
    - [originalLanguage, video, director, actors, actorsCharacter]
        - A pesar de que manifiestan insights cualitativos, su variabilidad y naturaleza categórica complican su uso directo en el clustering sin una transformación adecuada.

- Variables irrelevantes para clustering
    - [id, originalTitle, title, homePage, releaseDate (depende del contexto)]
        - Pues son id's o URLs que no aportan información agrupable sobre las características importantes de las películas.

- Variables útiles para clustering
    - [popularity, budget, revenue, runtime, genresAmount, productionCoAmount, productionCountriesAmount, voteCount, voteAvg, actorsPopularity, castWomenAmount, castMenAmount, releaseDate (el año)]
        - Ya que al ser variables cuantitativas, estas ofrecen una base sólida para agrupar las películas basándose en popularidad, aspectos financieros, contenido, involucrados y recepción del público.

### 2. Analice la tendencia al agrupamiento usando el estadístico de Hopkings y la VAT (Visual Assessment of cluster Tendency). Discuta sus resultados e impresiones.

### 3. Determine cuál es el número de grupos a formar más adecuado para los datos que está trabajando. Haga una gráfica de codo y explique la razón de la elección de la cantidad de clústeres con la que trabajará.

### 4. Utilice los algoritmos k-medias y clustering jerárquico para agrupar. Compare los resultados generados por cada uno.

### 5. Determine la calidad del agrupamiento hecho por cada algoritmo con el método de la silueta. Discuta los resultados.

In [ ]:
df['primary_genre'] = df['genres'].apply(lambda x: x.split('|')[0])

# Algoritmo de k-medias
kmeans = KMeans(n_clusters=5)
kmeans.fit(df)
kmeans_labels = kmeans.labels_

# Algoritmo de clustering jerárquico
agg_clustering = AgglomerativeClustering(n_clusters=5)
agg_clustering.fit(df)
agg_labels = agg_clustering.labels_

# Score de las siluetas 
silhouette_kmeans = silhouette_score(df, kmeans_labels)
silhouette_agg = silhouette_score(df, agg_labels)


print("Puntuación de la silueta para K-medias:", silhouette_kmeans)
print("Puntuación de la silueta para Clustering Jerárquico:", silhouette_agg)


### 6. Interprete los grupos basado en el conocimiento que tiene de los datos. Recuerde investigar las medidas de tendencia central de las variables continuas y las tablas de frecuencia de las variables categóricas pertenecientes a cada grupo. Identifique hallazgos interesantes debido a las agrupaciones y describa para qué le podría servir.